### Libraries

In [ ]:
#Including Libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import math

import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from scipy.stats import genpareto
from keras import losses

## DQN 

In [7]:
class DQNAgent:
    def __init__(self,gamma):        
        self.state_size = 2
        self.action_size = 5
        self.memory = deque(maxlen=20000)
        self.gamma = gamma   # discount rate
        self.epsilon = 1  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9999
        self.learning_rate = 0.001
        self.X_batch=[]
        self.Y_batch=[]
        self.target_weights=[]
        self.weights=[]
        self.cost_array=[]
        self.action_array=[]
        
        self.tau = 1 #target weights update rate        
              
        self.model        = self.create_model()
        self.target_model = self.create_model()
        
    def create_model(self):
        model = Sequential()
        model.add(Dense(12, input_dim=self.state_size, activation='linear'))
        model.add(Dense(24, activation='linear'))
        model.add(Dense(12, activation='linear'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=tf.keras.losses.Huber(),
                      optimizer=optimizers.RMSprop(lr=self.learning_rate, clipnorm=1))   
    
        return model
    
    def remember(self, state, action, cost, next_state):
        self.memory.append((state, action, cost, next_state))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(5)
        act_values = self.model.predict(state)
        return np.argmin(act_values)  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        state_array = np.vstack([x[0] for x in minibatch])
        self.action_array = np.array([x[1] for x in minibatch])
        self.cost_array = np.array([x[2] for x in minibatch])
        next_state_array = np.vstack([x[3] for x in minibatch])
        self.X_batch = state_array
        self.Y_batch = self.model.predict(state_array)

        Q_target = self.cost_array +self.gamma * np.amin(self.target_model.predict(next_state_array), axis=1) 
        self.Y_batch[np.arange(len(self.X_batch)), self.action_array] = Q_target

        self.model.fit(self.X_batch, self.Y_batch, epochs=1,verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def target_train(self):
        self.weights = self.model.get_weights()
        self.target_weights = self.target_model.get_weights()
        for i in range(len(self.target_weights)):
            self.target_weights[i] = self.weights[i] * self.tau + self.target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(self.target_weights)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)        

### Parameters

In [4]:
# NOAA sea level rise predictions
#input 'scale_parameter_gammadstrb'
B= 0.5;

#input 'shape_parameter_gammadstrb'
A= np.zeros((3,100))
A[0,:]=np.arange(11.2,12.388,0.012);# intermediate low rise case
A[1,:]=np.arange(13,35,0.22);# intermediate high rise case
A[2,:]= np.arange(14.6,88.6,0.74);# high rise case


# generalized pareto    
power_l=0.9;
power_s= 0.8;  
eta=250;
k= -0.1;
theta= 14;

#parameters for cost definition
beta=14 ; #coefficient of residents' investment decision y_n, residents', contribution of 14M $/yearly, 1M $ taken as unit
alpha=300; # 25m $ is the coefficient for investment cost

### Main Code

In [ ]:
%%time
a_g= 0.9; ## Government's cooperation index
a_r= 0.9; ## Residents' cooperation index
a=2; ## SLR scenarios; a=0,1, and 2, respectively for intermediate low, intermediate, and high sea level rise projections. 
s_norm=400,# normalozing s
l_norm=[580, 1190,2590];# normalozing s
scene= ['low','inter','high']

EPISODES= 1000000;
Final_Episodes=100; ## episodes used for final cost evaluation

years=100; #future to be considered in years 

agent = DQNAgent(a_g);
batch_size=1000;
state= np.zeros((100,2));
next_state= np.zeros((100,2));
convergence=[];
w00= np.zeros((2,12));
w01= np.zeros((12,24));
w02= np.zeros((24,12));
w03= np.zeros((12,5));
b00= np.zeros(12);
b01= np.zeros(24);
b02= np.zeros(12);
b03= np.zeros(5);

total_cost= 0; ## average cost over all episodes
final_cost= 0; ## average cost for last 10000 episodes(convergence expected by then)

for e in range(EPISODES):
	l= np.zeros(101);
	s= np.zeros(101);
	l[0]=100; # initial relative sea level
	s[0]=50; # initial infrastructure state  
	r=0; # sea level rise each year
	q= np.zeros(101);  ## yearly residents' decision score          
	p= np.zeros(101);  ## yearly sigmoid input for residents' decision           
	res= np.zeros(101); ## yearly residents' binary decision 
	x= np.zeros(100);   ##  yearly govenment's decision         
	z= np.zeros(100); ## yearly cost from nature
            
	for y in range(years):

		z[y]=genpareto.rvs(k, loc=theta, scale=eta* np.power(l[y],power_l)/np.power(s[y],power_s)); # genpareto, shape,k=-0.001,location,theta=0; scale,sigma         
               
		state[y,0]= (l[y]-l[0])/l_norm[a];             
		state[y,1]= (s[y]-s[0])/s_norm;             
                
		action = agent.act(np.reshape(state[y],(1,2)));
		x[y]= action; 
		next_state[y,0]= (l[y]-l[0]+A[a,y]/2)/l_norm[a];            
		next_state[y,1]= (s[y]-s[0]+action)/s_norm;  
		cost= alpha*x[y]-beta*res[y]+z[y];  
		total_cost=total_cost+ cost;
		if e> np.absolute(EPISODES-Final_Episodes-1):
			final_cost=final_cost+ cost;                        
		agent.remember(state[y], action, cost, next_state[y]) 
		if len(agent.memory) > batch_size:
			w0= agent.model.layers[0].get_weights()[0];
			w1= agent.model.layers[1].get_weights()[0];                        
			w2= agent.model.layers[2].get_weights()[0];                        
			w3= agent.model.layers[3].get_weights()[0];  
			b0= agent.model.layers[0].get_weights()[1];
			b1= agent.model.layers[1].get_weights()[1];                        
			b2= agent.model.layers[2].get_weights()[1];                        
			b3= agent.model.layers[3].get_weights()[1];                        
			conv=np.sum(np.power((w0-w00),2))+np.sum(np.power((w1-w01),2))+np.sum(np.power((w2-w02),2))+np.sum(np.power((w3-w03),2))+np.sum(np.power((b0-b00),2))+np.sum(np.power((b1-b01),2))+np.sum(np.power((b2-b02),2))+np.sum(np.power((b3-b03),2));
			convergence=  np.append(convergence,conv);
			w00=w0;
			w01=w1;
			w02=w2;
			w03=w3;
			b00=b0;
			b01=b1;
			b02=b2;
			b03=b3;                        
			agent.replay(batch_size); 
			if y % 30 == 1:       
				agent.target_train() 
                
		q[y+1]=a_r*(q[y]+ action/4 * z[y]); ## dividing the action by 4 because action can be o~3
		p[y+1]= 1/(1 + np.exp(-(q[y+1]-5)));
		res[y+1]= np.random.binomial(1, p[y+1]);  

		r= np.random.gamma(A[a,y],B);                                         
		l[y+1]= l[y]+r;                
		s[y+1]= s[y]+action; 
        
            
total_cost=total_cost/EPISODES;
final_cost=final_cost/Final_Episodes;
print("Total cost: {}, final cost: {}" .format(total_cost,final_cost))
